In [1]:
import pandas as pd
import json
import uuid
from fhir.resources.bundle import Bundle
from fhir.resources.patient import Patient
from fhir.resources.narrative import Narrative

# Load the CSV file
csv_file = 'Patient_resources.csv'
data = pd.read_csv(csv_file)

# Convert date_of_birth column to ISO 8601 format strings
data['date_of_birth'] = pd.to_datetime(data['date_of_birth']).dt.strftime('%Y-%m-%d')

# Create a list to hold Bundle entries
entries = []

# Process each row in the CSV

for index, row in data.iterrows():
    deceasedBoolean = row['living_status'].lower() == 'expired'  # Determine deceased status

    patient = Patient.construct(
        id=str(row["subject_id"]),
        identifier=[{"value": str(row["subject_id"])}],
        birthDate=row["date_of_birth"],
        gender=row["gender"],
        deceasedBoolean=deceasedBoolean
    )
    
    # Create a narrative for the patient
    narrative_text = f"<div xmlns='http://www.w3.org/1999/xhtml'>Patient record for {row['subject_id']}</div>"
    patient.text = Narrative.construct(status="generated", div=narrative_text)

    # Create a Bundle entry
    entry = {
        "fullUrl": f"urn:uuid:{str(uuid.uuid4())}",
        "resource": patient.dict(),
        "request": {
            "method": "POST",  # Use POST to create new resources
            "url": "Patient"   # Specify the resource type
        }
    }

    # Add the entry to the list
    entries.append(entry)

# Create a Bundle resource
bundle = Bundle.construct(
    type="transaction",  # Set Bundle type to transaction
    entry=entries
)

# Convert the Bundle to JSON
bundle_json = bundle.dict()

# Save the JSON to a file
output_file = 'fhir_patient_bundle_transaction_main_file.json'
with open(output_file, 'w') as f:
    json.dump(bundle_json, f, indent=2)

print(f"FHIR Patient Bundle with fullUrl has been saved to {output_file}")


FHIR Patient Bundle with fullUrl has been saved to fhir_patient_bundle_with_fullUrl_lowercase_transaction_part2.json


In [3]:

#Uploading it to the local HAPI FHIR server (R5 version)
import json
import requests

# Load the JSON data from the file
input_file_path = 'fhir_patient_bundle_transaction_main_file.json'
output_file_path = 'uploaded_resource_ids.json'

with open(input_file_path, 'r') as file:
    data = json.load(file)

# Define the HAPI FHIR server URL
url = 'http://localhost:8080/fhir'

# Make a POST request to upload the Bundle
response = requests.post(url, json=data, headers={'Content-Type': 'application/fhir+json'})

if response.status_code == 200:
    print("Successfully uploaded the JSON file to the HAPI FHIR server.")

    # Parse the response to extract resource IDs
    response_data = response.json()
    resource_ids = []

    for entry in response_data.get('entry', []):
        resource_id = entry.get('response', {}).get('location', '')
        resource_ids.append(resource_id)

    # Save the resource IDs to a new JSON file
    with open(output_file_path, 'w') as output_file:
        json.dump(resource_ids, output_file, indent=4)

    print(f"Resource IDs have been saved to {output_file_path}.")
else:
    print(f"Failed to upload the JSON file. Status code: {response.status_code}")
    print(response.text)


Successfully uploaded the JSON file to the HAPI FHIR server.
Resource IDs have been saved to uploaded_resource_ids.json.
